You have to work on the [University dataset](https://drive.google.com/drive/folders/1Hs3nRtK_F3h8eg59B4-TD1DEua6g8Klv?usp=sharing). It contains three different university rankings:
*  The Times Higher Education World University Ranking, shortly *Times*,
*  the Academic Ranking of World Universities, shortly *Shanghai*,
*  the Center for World University Rankings, shortly *cwur*.

### Notes

1.    It is mandatory to use GitHub for developing the project.
1.    The project must be a jupyter notebook.
1.    There is no restriction on the libraries that can be used, nor on the Python version.
1.    All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com).

In [1]:
import pandas as pd
import numpy as np
import re
import math

First of all we open the three datasets of the university rankings and create three corresponding dataframes.
We use the _head( )_ function to check if the dataframe has been defined correctly.

In [2]:
cwur=pd.read_csv("cwurData.csv")
cwur.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


In [3]:
shanghai=pd.read_csv("shanghaiData.csv")
shanghai.head()

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005
1,2,University of Cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005
2,3,Stanford University,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005
3,4,"University of California, Berkeley",3,72.8,71.8,76.0,69.4,73.9,72.2,52.7,2005
4,5,Massachusetts Institute of Technology (MIT),4,70.1,74.0,80.6,66.7,65.8,64.3,53.0,2005


In [4]:
times=pd.read_csv("timesData.csv")
times.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33%,37 : 63,2011
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,"15,596",7.8,22%,42 : 58,2011
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,-,94.2,"7,929",8.4,27%,45 : 55,2011


### 1. For each university, extract from the *times* dataset the most recent and the least recent data, obtaining two separate dataframes

We apply the function `idxmax()` to the field *year* with the function `groupby`: in this way, for every university, we obtain the index of the row with the maximum year. Then, to access the rows of the *times* dataset related to the data identified by the function `idxmax()`, we use the `loc` function.

In [5]:
times_most=times.loc[times.groupby('university_name').year.idxmax()]
times_most.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016


We proceed in analogy to what we have just done to find the rows related to the least recent data for each university.

In [6]:
times_least=times.loc[times.groupby('university_name').year.idxmin()]
times_least.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
501,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,-,"17,422",15.9,15%,48 : 52,2012
502,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,-,"16,099",24.2,17%,32 : 68,2012
166,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,"23,895",13.6,14%,54 : 46,2011
476,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,-,"9,252",19.2,18%,48 : 52,2012


### 2. For each university, compute the improvement in `income` between the least recent and the most recent data points

To calculate the improvement in income, we need to work on both the *times_most* table and the *times_least* table: we use the merge function to build the *improvement* table, which contains all the data we need. Moreover, where the income data is missing, we replace the symbol "-" with the constant value *NaN*.

In [7]:
improvement=pd.merge(times_most[['university_name','income','year']], times_least[['university_name','income','year']], on='university_name', suffixes=['_most', '_least'])
improvement['income_most']=improvement['income_most'].replace('-', np.NaN)
improvement['income_least']=improvement['income_least'].replace('-', np.NaN)
improvement.head()

,university_name,income_most,year_most,income_least,year_least
0,AGH University of Science and Technology,NaN,2016,NaN,2016
1,Aalborg University,43.7,2016,36.4,2012
2,Aalto University,61.6,2016,61.9,2012
3,Aarhus University,68.3,2016,61.5,2011
4,Aberystwyth University,31.3,2016,35.5,2012


In order to create the *improvement (%)* column, we cast the income data to float. We then decided to calculate the improvement in income as the difference between the most recent year's income and the least recent year's income, normalized to the least recent year's income and expressed as a percentage. For example, if the income has doubled, our result must be 100%.

In [8]:
improvement["income_most"]=improvement["income_most"].astype(float)
improvement["income_least"]=improvement["income_least"].astype(float)
improvement["improvement (%)"]=round((improvement["income_most"]-improvement["income_least"])/improvement["income_least"]*100,2)
improvement

,university_name,income_most,year_most,income_least,year_least,improvement (%)
0,AGH University of Science and Technology,NaN,2016,NaN,2016,NaN
1,Aalborg University,43.7,2016,36.4,2012,20.05
2,Aalto University,61.6,2016,61.9,2012,-0.48
3,Aarhus University,68.3,2016,61.5,2011,11.06
4,Aberystwyth University,31.3,2016,35.5,2012,-11.83
...,...,...,...,...,...,...
813,École Normale Supérieure,37.1,2016,30.7,2011,20.85
814,École Normale Supérieure de Lyon,31.7,2016,26.1,2011,21.46
815,École Polytechnique,82.3,2016,NaN,2011,NaN
816,École Polytechnique Fédérale de Lausanne,65.4,2016,38.0,2011,72.11


### 3. Find the university with the largest increase computed in the previous point

We apply the `idxmax()` function to the field *improvement (%)* in order to obtain the index of the university with the maximum improvement. Then, to get which university has been identified by `idxmax()`, we use the `loc` function.

In [9]:
improvement.loc[improvement['improvement (%)'].idxmax()]

university_name    TU Dresden
income_most              99.7
year_most                2016
income_least             31.9
year_least               2012
improvement (%)        212.54
Name: 428, dtype: object

### 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for *Aarhus University* the value is 122-73=49). Notice that some rankings are expressed as a range

To obtain dataframes containing only the most recent data of each university, we proceed in analogy to what we did in the first exercise for the *times* dataset.

In the case of the *cwur* dataset, we rename the *institution* field to conform it to the *university_name* fields of the other two datasets.

In [10]:
cwur_new=cwur.loc[cwur.groupby('institution').year.idxmax()]
cwur_new.rename(columns={'institution':'university_name'}, inplace=True)
cwur_new.head()

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
1981,782,AGH University of Science and Technology,Poland,4,267,279,218,708,891,511,896.0,398,44.26,2015
1764,565,Aalborg University,Denmark,5,367,439,218,596,660,287,520.0,401,44.59,2015
1620,421,Aalto University,Finland,4,367,224,218,336,403,645,424.0,246,45.10,2015
1321,122,Aarhus University,Denmark,2,310,165,106,91,81,101,95.0,154,49.65,2015
2013,814,Aberystwyth University,United Kingdom,59,367,567,218,842,552,511,754.0,607,44.22,2015


In [11]:
shanghai_new=shanghai.loc[shanghai.groupby('university_name').year.idxmax()]
shanghai_new.head()

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
4697,301-400,Aalborg University,5,NaN,0.0,0.0,11.2,4.6,30.4,16.8,2015
4797,401-500,Aalto University,4-6,NaN,0.0,0.0,0.0,10.4,33.4,17.0,2015
4469,73,Aarhus University,2,27.3,11.5,22.1,12.3,25.8,51.8,31.0,2015
4497,101-150,Aix Marseille University,5-6,NaN,13.6,0.0,15.2,20.3,50.1,27.6,2015
3115,102-150,Aix-Marseille University,4-6,NaN,15.6,0.0,16.1,22.4,44.3,15.3,2011


In [12]:
times_new=times_most
times_new.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016


We observe that there are names of universities written differently but which refer to the same authority. For example in the database *Shanghai_new* we have "Aix Marseille University" and "Aix-Marseille University" or in the database *Chwur_new* we have "University of Bordeaux" and "University of Bordeaux I", as shown below.

In [13]:
cwur_new[cwur_new['university_name'].str.contains('University of Bordeaux')]

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
1498,299,University of Bordeaux,France,14,354,567,218,291,265,234,251.0,297,45.96,2015
631,432,University of Bordeaux I,France,17,355,478,210,327,228,406,361.0,426,45.41,2014
602,403,University of Bordeaux II,France,15,355,478,210,440,385,363,322.0,426,45.54,2014


We define a function that is able to normalize the university names within the three dataframes (shanghai_new, times_new and cwur_new). We use *school_and_country_table* as the reference database.

In [14]:
school_and_country=pd.read_csv("school_and_country_table.csv")
school_and_country.rename(columns={'school_name':'university_name_normalized'},inplace=True)
school_and_country.head()

,university_name_normalized,country
0,Harvard University,United States of America
1,California Institute of Technology,United States of America
2,Massachusetts Institute of Technology,United States of America
3,Stanford University,United States of America
4,Princeton University,United States of America


We create a list containing all the names of the universities taken from the *school_and_country* dataset.

In [15]:
university_name_normalized=[]
for university in school_and_country['university_name_normalized']:
    if university not in university_name_normalized:
        university_name_normalized.append(university)

To create the functions to normalize the names of universities we use functions contained in the *fuzzywuzzy* library and in particular in its module called *process*.

In [16]:
from fuzzywuzzy import process

* The funcion *name_normalized* takes as input the name of a university and returns, among all the universities in the *university_name_normalized* list, the one with the most similar name, based on the scores assigned by the *extractOne* method of the *fuzzywuzzy* library:

In [17]:
def name_normalized(university):
    return(process.extractOne(university, university_name_normalized)[0])

* The function *score_match* works the same as *name_normalized*, but it returns the score attributed to the university within the *university_name_normalized* list with the name closest to the university provided in input:

In [18]:
def score_match(university):
    return(int(process.extractOne(university, university_name_normalized)[1]))

* The function *perfect_match* iterates through all the names of the universities in the dataframe *school_and_country* and returns the string provided in input, if it exactly matches the name of a university in the dataframe:

In [19]:
def perfect_match(string):
    for university in school_and_country['university_name_normalized']:
        if string == university:
            return string

* The function *complete_procedure* takes a dataframe as input and uses the three functions described above:
    * a field called *university_name_normalized* is created inside the dataframe given in input, applying the *perfect_match* function to the *university_name* field;
    * a new DataFrame called *good* is created as a copy of the table given in input, keeping only the rows where the *university_name_normalized* field has non-null values. A field called *score* is added to the table *good*; *score* contains the value "100" for each row, as the *good* table has been created with the universities that have found a perfect match with the universities contained in the *school_and_country* dataframe;
    * another new DataFrame, called *dirty*, is created as the complement of *good* as it contains the universities that have not found a perfect match. In this case the *university_name_normalized* field is created by applying the *name_normalized* function to the *university_name* field. The *score* column is filled using the *score_match* function;
    * *complete_procedure* outputs the DataFrame obtained from the concatenation between the *good* and *dirty* dataframes.

In [20]:
def complete_procedure(table):
    table['university_name_normalized'] = table['university_name'].apply(perfect_match)
    good=table[table['university_name_normalized'].isnull()==False].copy()
    good['score']=100
    dirty=table[table['university_name_normalized'].isnull()==True].copy()
    dirty['university_name_normalized'] = dirty['university_name'].apply(name_normalized)
    dirty['score'] = dirty['university_name'].apply(score_match)
    return(pd.concat([good, dirty]))

* The function *delete* takes as input a list of indices, a DataFrame and an empty list, which must be filled with incorrect university names. The *delete* function for each element of the list of indices, that is, for each index, assigns the name of the university corresponding to that index to the variable *uni*. If the name of the university that has been assigned to *uni* has not already been added to the input list, it is added. The result of this function is therefore to obtain a list of wrong university names, without repetition:

In [21]:
def delete(indices, table, li):
    for el in indices:
        uni=table.loc[el]['university_name']
        if uni not in li:
            li.append(uni)

We create empty lists that will contain the wrong university names:

In [22]:
wrong_names_shanghai=[]
wrong_names_cwur=[]

We give the DataFrame containing the most recent data of the Shanghai dataset as input to the function *complete_procedure*:

In [23]:
shanghai_new=complete_procedure(shanghai_new)
shanghai_new.head()

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,university_name_normalized,score
4697,301-400,Aalborg University,5,NaN,0.0,0.0,11.2,4.6,30.4,16.8,2015,Aalborg University,100
4797,401-500,Aalto University,4-6,NaN,0.0,0.0,0.0,10.4,33.4,17.0,2015,Aalto University,100
4469,73,Aarhus University,2,27.3,11.5,22.1,12.3,25.8,51.8,31.0,2015,Aarhus University,100
3115,102-150,Aix-Marseille University,4-6,NaN,15.6,0.0,16.1,22.4,44.3,15.3,2011,Aix-Marseille University,100
4798,401-500,Aristotle University of Thessaloniki,2,NaN,0.0,0.0,4.9,1.5,34.0,16.8,2015,Aristotle University of Thessaloniki,100


At this point we proceed with a manual cleaning operation to correct any errors made by *complete_procedure*.

We start by comparing the fields *university_name* and *university_name_normalized* of all the rows that were given a score lower than or equal to 86 (therefore very low). We observe that *university_name* and *university_name_normalized* in these cases never refer to the same university.

In [24]:
shanghai_new[shanghai_new['score']<=86].head(10)[['university_name', 'university_name_normalized']]

,university_name,university_name_normalized
4498,Baylor College of Medicine,ETH Zurich – Swiss Federal Institute of Techno...
4698,Beihang University,"University of California, Berkeley"
4599,Beijing Normal University,Yale University
3416,Beijing University of Aeronautics and Astronau...,Harvard University
4699,Brigham Young University,Yale University
4809,Catholic University of Korea,Harvard University
3616,Catholic University of Leuven,Harvard University
4548,Catholic University of Louvain,Harvard University
4700,Central South University,Yale University
4705,City University of New York City College,Harvard University


We then give all the rows of the *shanghai_new* table that have scores less than or equal to 86 as input to the *delete* function. In this way we fill the *wrong_names_shanghai* list with all the *university_names* that do not match the *university_name_normalized* with which they have been paired.

In [25]:
delete(shanghai_new.index[shanghai_new['score']<=86], shanghai_new, wrong_names_shanghai)

The previous step was used to keep track of the rows containing the incorrect results of the *complete_procedure* function. We then remove these rows permanently from the *shanghai_new* DataFrame using the `drop` function`.

In [26]:
shanghai_new.drop(shanghai_new.index[shanghai_new['score']<=86], inplace=True)

This analysis procedure which includes:
* identification of the wrong pairings *university_name* and *university_name_normalized*;
* registration of the *university_name* in question in the list of wrong names;
* elimination of the rows containing those *university_name* from the DataFrame

is manually repeated for all scores above 86.

In [27]:
shanghai_new[shanghai_new['score']==87][['university_name', 'university_name_normalized']]

,university_name,university_name_normalized
423,Loyola University Chicago,University of Chicago
438,Northern Arizona University,University of Arizona
3358,Sao Paulo State University,São Paulo State University (UNESP)
4763,University of Bochum,Ruhr University Bochum
4669,University of Leipzig,University of Liège
2487,University of Lille 1,University of Liège
4885,University of Rostock,University of Oslo


In [28]:
delete([4669,2487,4885], shanghai_new, wrong_names_shanghai)

In [29]:
shanghai_new.drop([4669,2487,4885], inplace=True)

In [30]:
shanghai_new[shanghai_new['score']==88][['university_name', 'university_name_normalized']]

,university_name,university_name_normalized
2931,Gunma University,Hunan University
4528,University of Frankfurt,Goethe University Frankfurt
4773,University of Malaya,University of Calgary
4881,University of Regensburg,University of Gothenburg
494,University of Wroclaw,University of Warsaw
4596,Virginia Commonwealth University,University of Virginia
1508,West Virginia University,University of Virginia


In [31]:
delete([2931,4773,4881,494], shanghai_new, wrong_names_shanghai)

In [32]:
shanghai_new.drop([2931,4773,4881,494], inplace=True)

In [33]:
shanghai_new[shanghai_new['score']==89][['university_name', 'university_name_normalized']]

,university_name,university_name_normalized
3722,Dresden University of Technology,Brno University of Technology
4829,Nara Institute of Science and Technology,Gwangju Institute of Science and Technology
2490,University of Louisville,University of Seville
1991,University of Maine,University of Miami
3494,University of Sherbrooke,Université de Sherbrooke
3498,University of Tromso,University of Oslo
4543,University of Wageningen,University of Groningen


In [34]:
delete([3722,4829,2490,1991,3498,4543], shanghai_new, wrong_names_shanghai)

In [35]:
shanghai_new.drop([3722,4829,2490,1991,3498,4543], inplace=True)

In [36]:
shanghai_new[shanghai_new['score']==90][['university_name', 'university_name_normalized']]

,university_name,university_name_normalized
3425,Curtin University of Technology,Curtin University
4549,Erasmus University,Erasmus University Rotterdam
3430,Federal University of Sao Paulo,Federal University of São Carlos
4504,Indiana University Bloomington,Indiana University
4616,Indiana University-Purdue University at Indian...,Purdue University
4722,King Abdullah University of Science and Techno...,AGH University of Science and Technology
4624,Louisiana State University - Baton Rouge,Louisiana State University
925,Louisiana State University Health Sciences Center,Louisiana State University
4456,Pennsylvania State University - University Park,Pennsylvania State University
4458,Purdue University - West Lafayette,Purdue University


In [37]:
delete([4722,4749,4771,4674,4448,4879], shanghai_new, wrong_names_shanghai)

In [38]:
shanghai_new.drop([4722,4749,4771,4674,4448,4879], inplace=True)

In [39]:
shanghai_new[shanghai_new['score']==91][['university_name', 'university_name_normalized']]

,university_name,university_name_normalized
2462,"Royal Holloway, U. of London","Royal Holloway, University of London"


In [40]:
shanghai_new[shanghai_new['score']==92][['university_name', 'university_name_normalized']]

,university_name,university_name_normalized
4611,Ecole Normale Superieure - Lyon,École Normale Supérieure de Lyon
4867,University of Jena,University of Genoa
4583,University of Koeln,University of Kiel
4293,Vrije University Brussel,Vrije Universiteit Brussel


In [41]:
delete([4867,4583], shanghai_new, wrong_names_shanghai)

In [42]:
shanghai_new.drop([4867,4583], inplace=True)

In [43]:
shanghai_new[shanghai_new['score']==93][['university_name', 'university_name_normalized']]

,university_name,university_name_normalized
4814,Eotvos Lorand University,Eötvös Loránd University
4026,University Libre Bruxelles,Université Libre de Bruxelles
4884,University of Roma - Tor Vergata,University of Rome II – Tor Vergata


In [44]:
shanghai_new[shanghai_new['score']==95][['university_name', 'university_name_normalized']]
#shanghai_new[shanghai_new['score']==95][['university_name', 'university_name_normalized']].head(60)
#shanghai_new[shanghai_new['score']==95][['university_name', 'university_name_normalized']].tail(30)

,university_name,university_name_normalized
3876,Arizona State University - Tempe,Arizona State University
4804,Brunel University,Brunel University London
4307,Capital University of Medical Sciences,Capital Medical University
4808,Catholic University of Chile,Pontifical Catholic University of Chile
4704,China Medical University,"China Medical University, Taiwan"
...,...,...
4595,University of Wuerzburg,University of Würzburg
4501,Université libre de Bruxelles (ULB),Université Libre de Bruxelles
4613,Vrije Universiteit Brussel (VUB),Vrije Universiteit Brussel
3309,Washington State University - Pullman,Washington State University


In [45]:
delete([4693], shanghai_new, wrong_names_shanghai)

In [46]:
shanghai_new.drop([4693], inplace=True)

In [47]:
shanghai_new[shanghai_new['score']>=96][['university_name', 'university_name_normalized']].head(10)

,university_name,university_name_normalized
4697,Aalborg University,Aalborg University
4797,Aalto University,Aalto University
4469,Aarhus University,Aarhus University
3115,Aix-Marseille University,Aix-Marseille University
4798,Aristotle University of Thessaloniki,Aristotle University of Thessaloniki
4489,Arizona State University,Arizona State University
4799,Auburn University,Auburn University
4597,Autonomous University of Barcelona,Autonomous University of Barcelona
4598,Autonomous University of Madrid,Autonomous University of Madrid
902,Bangor University,Bangor University


At this point we did a duplicate analysis by defining a function called *duplicate_check*.

*duplicate_check* takes a table as input, then defines an empty list, called *li*, and a new DataFrame, called *duplicate*, to which it assigns the rows of the input table that have the value of the *university_name_normalized* field that is not null and duplicate. *duplicate_check* iterates through each normalized university name in the *duplicate* DataFrame and if it is not yet present in *li*, it appends it to *li*; finally, for each element of *li*, it prints the normalized university name and the corresponding *year* and *university_name* fields of the input table.

This procedure is used to study if the duplicates found are made up of the universities that have changed their name over time (in this case we delete the oldest data) or if they are a campus of the same university (if the name in the table is a campus and not the name of a university, it is deleted) or different universities.

In [48]:
def duplicate_check(table):
    li=[]
    duplicate=table[(table['university_name_normalized'].isnull()==False) & (table['university_name_normalized'].duplicated()==True)]
    for el in duplicate['university_name_normalized']:
        if el not in li:
            li.append(el)
    for i in range(0,len(li)):
        print(table[table['university_name_normalized'] == li[i]][['university_name','year', 'university_name_normalized']])

In [49]:
duplicate_check(shanghai_new)

               university_name  year university_name_normalized
3115  Aix-Marseille University  2011   Aix-Marseille University
4497  Aix Marseille University  2015   Aix-Marseille University
                       university_name  year university_name_normalized
4489          Arizona State University  2015   Arizona State University
3876  Arizona State University - Tempe  2013   Arizona State University
                             university_name  year  university_name_normalized
4806              Capital Medical University  2015  Capital Medical University
4307  Capital University of Medical Sciences  2014  Capital Medical University
                      university_name  year university_name_normalized
4606                Curtin University  2015          Curtin University
3425  Curtin University of Technology  2011          Curtin University
                                        university_name  year  \
4624           Louisiana State University - Baton Rouge  2015   
925   Louisi

1508          West Virginia University  2007     University of Virginia
                       university_name  year  university_name_normalized
4613  Vrije Universiteit Brussel (VUB)  2015  Vrije Universiteit Brussel
4293          Vrije University Brussel  2014  Vrije Universiteit Brussel
                            university_name  year   university_name_normalized
4793            Washington State University  2015  Washington State University
3309  Washington State University - Pullman  2011  Washington State University


In [50]:
delete([3115,3876,4307,3425,925,423,4825,3346,4828,438,3738,4616,3741,4130,4133,2567,3606,3994,3913,3758,3461,3853,3654,3358,3362,4653,4854,3382,3383,3900,3748,4765,3775,4364,3852,3783,2984,3790,4675,4534,3820,4250,4174,4277,3655,3860,3961,4278,2501,4026,4596,1508,4293,3309], shanghai_new, wrong_names_shanghai)

In [51]:
shanghai_new.drop([3115,3876,4307,3425,925,423,4825,3346,4828,438,3738,4616,3741,4130,4133,2567,3606,3994,3913,3758,3461,3853,3654,3358,3362,4653,4854,3382,3383,3900,3748,4765,3775,4364,3852,3783,2984,3790,4675,4534,3820,4250,4174,4277,3655,3860,3961,4278,2501,4026,4596,1508,4293,3309], inplace=True)

In [52]:
duplicate_check(shanghai_new)

In the case of the *times_new* DataFrame, the *perfect_match* function directly solved the problem of the wrong university names, so it was not necessary to use *complete_procedure* and carry out the subsequent manual analysis.

In [53]:
#times_new=complete_procedure(times_new)
times_new['university_name_normalized']=times_new['university_name'].apply(perfect_match)
times_new.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,university_name_normalized
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016,AGH University of Science and Technology
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016,Aalborg University
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016,Aalto University
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016,Aarhus University
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016,Aberystwyth University


In [54]:
duplicate_check(times_new)

For the *cwur_new* DataFrame we repeated the same analysis carried out for *shanghai_new*.

In [55]:
cwur_new=complete_procedure(cwur_new)
cwur_new.head()

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year,university_name_normalized
1981,782,AGH University of Science and Technology,Poland,4,267,279,218,708,891,511,896.0,398,100,2015,AGH University of Science and Technology
1764,565,Aalborg University,Denmark,5,367,439,218,596,660,287,520.0,401,100,2015,Aalborg University
1620,421,Aalto University,Finland,4,367,224,218,336,403,645,424.0,246,100,2015,Aalto University
1321,122,Aarhus University,Denmark,2,310,165,106,91,81,101,95.0,154,100,2015,Aarhus University
2013,814,Aberystwyth University,United Kingdom,59,367,567,218,842,552,511,754.0,607,100,2015,Aberystwyth University


In [56]:
cwur_new[cwur_new['score']<=88].head(10)[['university_name', 'university_name_normalized']]

,university_name,university_name_normalized
2164,Ain Shams University,"University of California, Berkeley"
1849,Albany Medical College,"University of Maryland, College Park"
2050,"All India Institute of Medical Sciences, New D...",University of Cambridge
1924,Ankara University,Andhra University
1878,Banaras Hindu University,Yale University
1320,Baylor College of Medicine,ETH Zurich – Swiss Federal Institute of Techno...
1766,Baylor University,Bangor University
1904,Beihang University,"University of California, Berkeley"
1159,Beijing Jiaotong University,Harvard University
1595,Beijing Normal University,Yale University


In [57]:
delete(cwur_new.index[cwur_new['score']<=88], cwur_new, wrong_names_cwur)

In [58]:
cwur_new.drop(cwur_new.index[cwur_new['score']<=88], inplace=True)

In [59]:
cwur_new[cwur_new['score']==89][['university_name', 'university_name_normalized']]

,university_name,university_name_normalized
1573,Darmstadt University of Technology,Warsaw University of Technology
1409,Dresden University of Technology,Brno University of Technology
2197,Federal University of Ceará,Federal University of Lavras
2026,Nagoya Institute of Technology,Georgia Institute of Technology
1746,Nara Institute of Science and Technology,Gwangju Institute of Science and Technology
1911,San Francisco State University,University of San Francisco
1187,Shanxi University,Shanghai University
2162,Southern Medical University,Southern Federal University
1761,University Lille 1: Sciences and Technologies,Lille 1 University – Science and Technology
2121,University of Basilicata,University of Brasília


In [60]:
delete(cwur_new.index[(cwur_new['score']==89) & (cwur_new['university_name']!='University of Sherbrooke') & (cwur_new['university_name'] != 'University Lille 1: Sciences and Technologies')], cwur_new, wrong_names_cwur)

In [61]:
cwur_new.drop(cwur_new.index[(cwur_new['score']==89) & (cwur_new['university_name']!='University of Sherbrooke') & (cwur_new['university_name'] != 'University Lille 1: Sciences and Technologies')], inplace=True)

In [62]:
cwur_new[cwur_new['score']==90][['university_name', 'university_name_normalized']]

,university_name,university_name_normalized
1354,Albert Ludwig University of Freiburg,University of Freiburg
1846,Binghamton University,"Binghamton University, State University of New..."
1470,Colorado State University - Fort Collins,Colorado State University
2082,Hefei University of Technology,Delft University of Technology
1305,Indiana University - Bloomington,Indiana University
1363,Indiana University-Purdue University Indianapolis,Purdue University
2154,Jinan University,Inha University
1434,Julius-Maximilian's University of Würzburg,University of Würzburg
1903,King Abdullah University of Science and Techno...,AGH University of Science and Technology
1970,Kurume University,Umeå University


In [63]:
delete([2082,2154,1903,1970,2167,1972,2109,1742,2058,1872,1545,1889,2078,1543,2163,1980], cwur_new, wrong_names_cwur)

In [64]:
cwur_new.drop([2082,2154,1903,1970,2167,1972,2109,1742,2058,1872,1545,1889,2078,1543,2163,1980], inplace=True)

In [65]:
cwur_new[cwur_new['score']==91][['university_name', 'university_name_normalized']]

,university_name,university_name_normalized
1936,Kindai University,Kinki University
1809,"Putra University, Malaysia",Universiti Putra Malaysia
2044,Sofia University,Sophia University
2123,South China Normal University,East China Normal University
1944,University of Calabria,University of Cagliari
1908,University of Cantabria,University of Catania
1851,University of Los Andes (Colombia),"University of the Andes, Colombia"


In [66]:
delete([2123,1944,1908], cwur_new, wrong_names_cwur)

In [67]:
cwur_new.drop([2123,1944,1908], inplace=True)

In [68]:
cwur_new[cwur_new['score']==92][['university_name', 'university_name_normalized']]

,university_name,university_name_normalized
1219,Swiss Federal Institute of Technology in Zurich,ETH Zurich – Swiss Federal Institute of Techno...
2134,University of Jaén,University of Jordan
1527,University of Jena,University of Genoa
2112,University of North Dakota,University of South Dakota
2056,University of Siegen,University of Siena
2143,Wrocław University of Technology,Warsaw University of Technology


In [69]:
delete([2134,1527,2112,2056,2143], cwur_new, wrong_names_cwur)

In [70]:
cwur_new.drop([2134,1527,2112,2056,2143], inplace=True)

In [71]:
cwur_new[cwur_new['score']==93][['university_name', 'university_name_normalized']]

,university_name,university_name_normalized
2173,Federal University of Santa Maria,Federal University of Santa Catarina
2086,Kyoto Institute of Technology,Tokyo Institute of Technology
1979,Mie University,Umeå University
2003,Nanjing University of Science and Technology,AGH University of Science and Technology
2139,University of Almería,University of Alberta


In [72]:
delete([2173,2086,1979,2003,2139], cwur_new, wrong_names_cwur)

In [73]:
cwur_new.drop([2173,2086,1979,2003,2139], inplace=True)

In [74]:
cwur_new[cwur_new['score']==94][['university_name', 'university_name_normalized']]

,university_name,university_name_normalized
1926,Toho University,Tohoku University


In [75]:
delete([1926], cwur_new, wrong_names_cwur)

In [76]:
cwur_new.drop([1926], inplace=True)

In [77]:
cwur_new[cwur_new['score']==95][['university_name', 'university_name_normalized']]
#cwur_new[cwur_new['score']==95].head(60)[['university_name', 'university_name_normalized']]
#cwur_new[cwur_new['score']==95].tail(10)[['university_name', 'university_name_normalized']]

,university_name,university_name_normalized
2089,Adam Mickiewicz University in Poznań,Adam Mickiewicz University
2176,Beijing University of Technology,University of Science and Technology Beijing
1671,Brunel University,Brunel University London
2005,Chungbuk National University,Chonbuk National University
2133,Federal University of Paraná,Federal University of Paraná (UFPR)
...,...,...
2081,University of Western Sydney,University of Sydney
1579,"Washington State University, Pullman",Washington State University
1190,Zhejiang Normal University,Zhejiang University
1236,École normale supérieure - Paris,École Normale Supérieure


In [78]:
delete([1902], cwur_new, wrong_names_cwur)

In [79]:
cwur_new.drop([1902], inplace=True)

In [80]:
cwur_new[cwur_new['score']>=96][['university_name', 'university_name_normalized']]

,university_name,university_name_normalized
1981,AGH University of Science and Technology,AGH University of Science and Technology
1764,Aalborg University,Aalborg University
1620,Aalto University,Aalto University
1321,Aarhus University,Aarhus University
2013,Aberystwyth University,Aberystwyth University
...,...,...
1362,Université libre de Bruxelles,Université Libre de Bruxelles
1475,Wageningen University and Research Centre,Wageningen University and Research Center
1251,Washington University in St. Louis,Washington University in St Louis
1588,Xi'an Jiaotong University,Xi’an Jiaotong University


In [81]:
duplicate_check(cwur_new)

                                   university_name  year  \
1729  University of Science and Technology Beijing  2015   
2176              Beijing University of Technology  2015   

                        university_name_normalized  
1729  University of Science and Technology Beijing  
2176  University of Science and Technology Beijing  
                   university_name  year   university_name_normalized
1621   Chonbuk National University  2015  Chonbuk National University
2005  Chungbuk National University  2015  Chonbuk National University
                      university_name  year university_name_normalized
1331          University of São Paulo  2015    University of São Paulo
1788  Federal University of São Paulo  2015    University of São Paulo
              university_name  year university_name_normalized
1752         Hunan University  2015           Hunan University
2171  Hunan Normal University  2015           Hunan University
                               university_name  

In [82]:
delete([2176,2005,1788,2171,1824,1773,1798,2085,2063,2094,2009,2070,1363,1997,2087,2168,2175,1345,1628,631,602,1593,1374,2024,2083,1382,1753,1947,725,440,1675,1862,2081,1190,2183,1920], cwur_new, wrong_names_cwur)

In [83]:
cwur_new.drop([2176,2005,1788,2171,1824,1773,1798,2085,2063,2094,2009,2070,1363,1997,2087,2168,2175,1345,1628,631,602,1593,1374,2024,2083,1382,1753,1947,725,440,1675,1862,2081,1190,2183,1920], inplace=True)

In [84]:
duplicate_check(cwur_new)

After normalizing the university names in all three DataFrames, we use the `merge` function to get a single DataFrame with all the data.

In [85]:
all_ranking_partial = pd.merge(shanghai_new, times_new, on = 'university_name_normalized', suffixes=('_shanghai', '_times'))
all_ranking_partial

,world_rank_shanghai,university_name_shanghai,national_rank,total_score_shanghai,alumni,award,hici,ns,pub,pcp,...,international,research,citations,income,total_score_times,num_students,student_staff_ratio,international_students,female_male_ratio,year_times
0,301-400,Aalborg University,5,NaN,0.0,0.0,11.2,4.6,30.4,16.8,...,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016
1,401-500,Aalto University,4-6,NaN,0.0,0.0,0.0,10.4,33.4,17.0,...,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016
2,73,Aarhus University,2,27.3,11.5,22.1,12.3,25.8,51.8,31.0,...,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016
3,401-500,Aristotle University of Thessaloniki,2,NaN,0.0,0.0,4.9,1.5,34.0,16.8,...,36.6,15.0,29.5,33.6,-,"46,288",22.2,9%,52 : 48,2016
4,93,Arizona State University,47,24.5,0.0,20.0,22.2,25.5,42.6,19.1,...,31.9,38.1,84.6,32.0,49.7,"83,236",29.9,9%,50 : 50,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,101-150,Université libre de Bruxelles (ULB),3,NaN,19.9,26.6,11.3,11.8,32.7,30.4,...,84.5,28.3,47.9,39.1,-,"23,819",26.1,32%,54 : 46,2016
492,201-300,Vrije Universiteit Brussel (VUB),5-6,NaN,13.6,0.0,5.0,10.0,31.3,26.9,...,64.5,24.5,56.8,77.1,-,"9,020",17.1,16%,55 : 45,2016
493,32,Washington University in St. Louis,24,37.4,22.9,24.9,40.7,41.8,50.5,26.0,...,44.9,55.6,99.2,33.5,67.0,"12,528",5.7,17%,NaN,2016
494,201-300,Western University (The University of Western ...,8-16,NaN,0.0,0.0,5.1,15.0,41.9,18.0,...,73.6,32.4,59.0,59.7,-,"27,387",20.7,16%,54 : 46,2016


In [86]:
all_ranking = pd.merge(all_ranking_partial, cwur_new, on = 'university_name_normalized')[["university_name_normalized", "world_rank_times", "world_rank_shanghai", "world_rank"]]
all_ranking.head()

,university_name_normalized,world_rank_times,world_rank_shanghai,world_rank
0,Aalborg University,201-250,301-400,565
1,Aalto University,251-300,401-500,421
2,Aarhus University,=106,73,122
3,Aristotle University of Thessaloniki,601-800,401-500,459
4,Arizona State University,189,93,97


In some cases, the scores assigned to universities are expressed as intervals. To handle these cases, we use regular expressions and define the *average_value* function.

*average_value* takes a string as input (the string represents the score of each university). If the score is made up of a certain number of digits, followed by a character other than a digit and subsequently by other digits, it means that it is an interval and in this case the function returns its average value, which we consider as score. In the cases where the score is composed only of digits or of a single group of digits followed or preceded by symbols, the function returns the digits, which we will use as score.

In [87]:
interval=re.compile('(?P<num1>\d*)(\D*)(?P<num2>\d*)')

def average_value(string):
    expression=interval.search(string)
    num1=expression.group('num1')
    num2=expression.group('num2')
    if (num1!='') and (num2!=''):
        return (int(num1)+int(num2))/2
    else:
        if num1!='':
            return int(num1)
        if num2!='':
            return int(num2)
        return np.NaN

To complete the exercise, we define the *max_difference* variable as the difference between the maximum score and the minimum score between the three scores assigned for each university by the three different rankings. To do this we use the `zip()` function to aggregate the *world_rank* fields of each of the three rankings. To the rankings in which some scores were expressed as an interval (*shanghai* and *times*) we apply the *average_value* function defined previously. We therefore create a new field within the *all_ranking* table called *max_difference*, which contains the differences calculated as just explained. These differences were then rounded to the nearest major integer, using the `ceil` function of `math` library.

In [88]:
max_difference=[max(a,b,c)-min(a,b,c) for (a,b,c) in zip(all_ranking['world_rank'], all_ranking['world_rank_shanghai'].apply(average_value), all_ranking['world_rank_times'].apply(average_value))]
all_ranking['max_difference']=max_difference
all_ranking['max_difference']=all_ranking['max_difference'].apply(math.ceil)
all_ranking

,university_name_normalized,world_rank_times,world_rank_shanghai,world_rank,max_difference
0,Aalborg University,201-250,301-400,565,340
1,Aalto University,251-300,401-500,421,175
2,Aarhus University,=106,73,122,49
3,Aristotle University of Thessaloniki,601-800,401-500,459,250
4,Arizona State University,189,93,97,96
...,...,...,...,...,...
483,Université Libre de Bruxelles,351-400,101-150,163,250
484,Vrije Universiteit Brussel,301-350,201-300,280,75
485,Washington University in St Louis,=60,32,52,28
486,University of Western Ontario,201-250,201-300,133,118


### 5. Consider only the most recent data point of the *times* dataset. Compute the number of male and female students for each country.

In the *times_most* dataset we can find the `female_male_ratio` column which shows the number of females and the number of males, separated by the symbol `:`.
We create two new columns in the *times_most* dataset by splitting the `famale_male_ratio` column in order to obtain the percentage of females and the percentage of males for each row.

In [89]:
times_most[['percentage_of_females','percentage_of_males']]=(times_most.female_male_ratio.str.split(" : ",expand=True))
times_most['percentage_of_females']=pd.to_numeric(times_most['percentage_of_females'], errors='coerce') # errors='coerce' means that invalid parsing will be set as NaN.
times_most['percentage_of_males']=pd.to_numeric(times_most['percentage_of_males'], errors='coerce')
times_most['percentage_of_females']=(times_most['percentage_of_females'])/100
times_most['percentage_of_males']=(times_most['percentage_of_males'])/100
times_most.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,university_name_normalized,percentage_of_females,percentage_of_males
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016,AGH University of Science and Technology,NaN,NaN
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016,Aalborg University,0.48,0.52
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016,Aalto University,0.32,0.68
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016,Aarhus University,0.54,0.46
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016,Aberystwyth University,0.48,0.52


In [90]:
max_difference=[max(a,b,c)-min(a,b,c) for (a,b,c) in zip(all_ranking['world_rank'], all_ranking['world_rank_shanghai'].apply(average_value), all_ranking['world_rank_times'].apply(average_value))]
all_ranking['max_difference']=max_difference
all_ranking['max_difference']=all_ranking['max_difference'].apply(math.ceil)
all_ranking

,university_name_normalized,world_rank_times,world_rank_shanghai,world_rank,max_difference
0,Aalborg University,201-250,301-400,565,340
1,Aalto University,251-300,401-500,421,175
2,Aarhus University,=106,73,122,49
3,Aristotle University of Thessaloniki,601-800,401-500,459,250
4,Arizona State University,189,93,97,96
...,...,...,...,...,...
483,Université Libre de Bruxelles,351-400,101-150,163,250
484,Vrije Universiteit Brussel,301-350,201-300,280,75
485,Washington University in St Louis,=60,32,52,28
486,University of Western Ontario,201-250,201-300,133,118


To get the total numbers of girls and boys from their percentages we need the total number of students. The latter is expressed separating the thousands with a comma, which we remove.

In [91]:
times_most['num_students']=times_most['num_students'].str.replace(',', '')
times_most['num_students']=pd.to_numeric(times_most['num_students'], errors = 'coerce')
times_most.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,university_name_normalized,percentage_of_females,percentage_of_males
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,35569.0,17.0,1%,-,2016,AGH University of Science and Technology,NaN,NaN
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,17422.0,15.9,15%,48 : 52,2016,Aalborg University,0.48,0.52
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099.0,24.2,17%,32 : 68,2016,Aalto University,0.32,0.68
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016,Aarhus University,0.54,0.46
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,9252.0,19.2,18%,48 : 52,2016,Aberystwyth University,0.48,0.52


We add the *females* and *males* fields to the *times_most* DataFrame, multiplying the total number of students by the percentage of females and males respectively.

In [92]:
times_most['females']=round((times_most['num_students'])*(times_most['percentage_of_females']))
times_most['males']=round((times_most['num_students'])*(times_most['percentage_of_males']))
times_most.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,university_name_normalized,percentage_of_females,percentage_of_males,females,males
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,35569.0,17.0,1%,-,2016,AGH University of Science and Technology,NaN,NaN,NaN,NaN
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,17422.0,15.9,15%,48 : 52,2016,Aalborg University,0.48,0.52,8363.0,9059.0
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099.0,24.2,17%,32 : 68,2016,Aalto University,0.32,0.68,5152.0,10947.0
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,54 : 46,2016,Aarhus University,0.54,0.46,12903.0,10992.0
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,9252.0,19.2,18%,48 : 52,2016,Aberystwyth University,0.48,0.52,4441.0,4811.0


We define the DataFrame *total_students* which contains, for each country, the total number of females, the total number of males and the total number of students, obtained as the sum of the first two.

In [93]:
total_students=pd.DataFrame(times_most.groupby('country')['females'].sum())
total_students['males']=pd.DataFrame(times_most.groupby('country')['males'].sum())
total_students['total_students']=(total_students['females'])+(total_students['males'])
total_students.head()

,females,males,total_students
country,,,
Argentina,67191.0,41182.0,108373.0
Australia,391736.0,321640.0,713376.0
Austria,68364.0,66113.0,134477.0
Bangladesh,21323.0,41393.0,62716.0
Belarus,20219.0,9084.0,29303.0


### 6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

We exclude from the calculation of the ratio the universities in which the number of men is equal to 0, because they would make the ratio infinite.
The problem is actually represented by a single university, *Ewha Womans University* which, as a university accessible only to women, is not a representative case within a study of the ratio *(number of females)* over *(number of males)* within each institution.

In [94]:
times_most['female_male_ratio']=(times_most['females'] / times_most[times_most['males']!=0]['males']).round(decimals=2)
times_most.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,university_name_normalized,percentage_of_females,percentage_of_males,females,males
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,35569.0,17.0,1%,NaN,2016,AGH University of Science and Technology,NaN,NaN,NaN,NaN
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,17422.0,15.9,15%,0.92,2016,Aalborg University,0.48,0.52,8363.0,9059.0
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,16099.0,24.2,17%,0.47,2016,Aalto University,0.32,0.68,5152.0,10947.0
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,14%,1.17,2016,Aarhus University,0.54,0.46,12903.0,10992.0
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,9252.0,19.2,18%,0.92,2016,Aberystwyth University,0.48,0.52,4441.0,4811.0


We check that the university with the highest ratio *(number of females)* over *(number of males)* has finite ratio:

In [95]:
times_most.loc[times_most['female_male_ratio'].idxmax()]['female_male_ratio']

3.55

We create a DataFrame containing only the required universities: those that have the ratio *(number of females)* over *(number of males)* lower than the average ratio.

In [96]:
mean_ratio=round(times_most['female_male_ratio'].mean(),2)
below_avg_ratio=pd.DataFrame(times_most[times_most['female_male_ratio'] < mean_ratio][['university_name','country','num_students','female_male_ratio']])
below_avg_ratio

,university_name,country,num_students,female_male_ratio
2003,Aalborg University,Denmark,17422.0,0.92
2056,Aalto University,Finland,16099.0,0.47
2105,Aberystwyth University,United Kingdom,9252.0,0.92
2406,Ajou University,South Korea,12706.0,0.49
2408,Alexandria University,Egypt,127431.0,0.85
...,...,...,...,...
2104,Zhejiang University,China,47508.0,0.69
1856,École Normale Supérieure,France,2400.0,0.85
2013,École Normale Supérieure de Lyon,France,2218.0,0.96
1904,École Polytechnique,France,2429.0,0.22


One last check: the university in the DataFrame *below_avg_ratio* with maximum ratio must have a ratio lower than the average ratio.

In [97]:
(below_avg_ratio.loc[below_avg_ratio['female_male_ratio'].idxmax()]['female_male_ratio'])<mean_ratio

True

### 7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).

We create a DataFrame called *below_avg_country* to which, for each country, we add the total number of students belonging to universities which have the ratio between the number of females and the number of males below the average.

In [98]:
below_avg_country=pd.DataFrame(below_avg_ratio.groupby('country')['num_students'].sum())
below_avg_country.rename(columns={'num_students':'students_uni_below_avg'}, inplace=True)
below_avg_country.head()

,students_uni_below_avg
country,
Australia,146235.0
Austria,61033.0
Bangladesh,62716.0
Brazil,278774.0
Canada,86779.0


We create a DataFrame called *below_over_total* using the `merge` function on the *below_avg_country* and *total_students* DataFrames (of which we only consider the column *total_students*). We add a field called *ratio* to the DataFrame just defined, calculating the required ratio.

In [99]:
below_over_total=pd.merge(below_avg_country,total_students['total_students'], on='country')
below_over_total['ratio']=round(below_over_total['students_uni_below_avg']/below_over_total['total_students'],2)
below_over_total.head()

,students_uni_below_avg,total_students,ratio
country,,,
Australia,146235.0,713376.0,0.20
Austria,61033.0,134477.0,0.45
Bangladesh,62716.0,62716.0,1.00
Brazil,278774.0,494251.0,0.56
Canada,86779.0,623628.0,0.14


### 8. Read the file `educational_attainment_supplementary_data.csv`, discarding any row without `country_name` or `series_name`

In [100]:
edu_attainment=pd.read_csv('educational_attainment_supplementary_data.csv')
edu_attainment.head()

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN


We check if there are rows where `country_name` or `series_name` are empty fields:

In [101]:
edu_attainment[(edu_attainment['country_name'].isnull()==True) | (edu_attainment['series_name'].isnull()==True)]

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
79050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79053,Data from database: Education Statistics: Educ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79054,Last Updated: 10/20/2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We remove the rows in which `country_nam` or `series_name` do not contain any values.

In [102]:
edu_attainment.dropna(subset=['country_name', 'series_name'], inplace=True)
edu_attainment.head()

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN


We repeat the previous check to see if the wanted rows have actually been removed:

In [103]:
edu_attainment[(edu_attainment['country_name'].isnull()==True) | (edu_attainment['series_name'].isnull()==True)]

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015


### 9. From `attainment` build a dataframe with the same data, but with 4 columns: `country_name`, `series_name`, `year`, `value`

To get the required DataFrame we use the `melt` function from `pandas`. We give the DataFrame *edu_attainment* that we want to have in the long format, the two columns we want to use for the identifying variables (*country_name* and *series_name*) and the name of the identifier column (*year*) as inputs to `melt`.

In [104]:
edu_attainment_long=pd.melt(edu_attainment, id_vars=['country_name', 'series_name'], var_name='year')
edu_attainment_long

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54
...,...,...,...,...
2134345,Zimbabwe,UIS: Percentage of population age 25+ with som...,2015,NaN
2134346,Zimbabwe,UIS: Percentage of population age 25+ with som...,2015,NaN
2134347,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2015,NaN
2134348,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2015,NaN


### 10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

**Problem regarding the normalization of the university names**: in exercise 4 we used sub-tables containing only the most recent data, but in this case we have to normalize the total DataFrame and the *complete_procedure* function used before does not give correct results.
We have therefore defined the *final_complete_procedure* function, which takes two elements as input: a table and a list of errors.

*final_complete_procedure*:
* deletes from the table provided in input all the rows that have the *university_name* field empty;
* creates a new column called *university_name_normalized* in the input table by applying the *perfect_match* function to the *university_name* field;
* creates the *good* DataFrame with the rows that have a non-null result of the previous *perfect_match* application and creates the *dirty* table with the rows resulting in a null *university_name_normalized*;
* for each *university_name* of the *dirty table*, if this is present in the list of errors provided in input, deletes the corresponding row from *dirty*;
* creates a new field in the *dirty* table, called *university_name_normalized*, applying the *name_normalized* function to the *university_name* field of the same table;
* creates an empty list called *find* and appends each *university_name* of the *dirty* table to that list;
* iterates through all the *university_name* of the *good* table and if they match a key of the dictionary *special_cases* (created thanks to the previous analysis), it assigns the value associated with that key to the *search* variable. If the value of *search* is contained in the *find* list, the university with the corresponding name is removed from the *good* table;
* creates an empty list called *save* and appends each *university_name* from the *good* table to that list;
* iterates through all the *university_name* of the *dirty* table and removes from *dirty* those that are also present in the *save* list;
* returns the DataFrame obtained by the concatenation of *good* and *dirty*.

In [105]:
special_cases={'Aix-Marseille University':'Aix Marseille University',
     'Norwegian University of Science and Technology':'Norwegian University of Science and Technology - NTNU',
     'State University of Campinas':'University of Campinas',
     'University of Melbourne':'The University of Melbourne',
     'University of New South Wales':'The University of New South Wales',
     'University of Pittsburgh':'University of Pittsburgh, Pittsburgh Campus'}

In [106]:
def final_complete_procedure(table, wrong_names_list):
    table.drop(table.index[table['university_name'].isnull()==True], inplace=True)
    table['university_name_normalized'] = table['university_name'].apply(perfect_match)
    good=table[table['university_name_normalized'].isnull()==False].copy()
    #good['score']=100
    dirty=table[table['university_name_normalized'].isnull()==True].copy()
    for uni in dirty['university_name']:
        if uni in wrong_names_list:
            dirty.drop(dirty.index[dirty['university_name']==uni], inplace=True)
    dirty['university_name_normalized'] = dirty['university_name'].apply(name_normalized)
    #dirty['score'] = dirty['university_name'].apply(score_match)
    find=[]
    for uni in dirty['university_name']:
        find.append(uni)
    for uni in good['university_name']:
        if uni in special_cases:
            search=special_cases[uni]
            if search in find:
                good.drop(good.index[good['university_name']==uni], inplace=True)
    save=[]
    for uni in good['university_name_normalized']:
        save.append(uni)
    for uni in dirty['university_name_normalized']:
        if uni in save:
            dirty.drop(dirty.index[dirty['university_name_normalized']==uni], inplace=True)
    return(pd.concat([good, dirty]))

We proceed with the normalization of the university names in the three different DataFrames, applying the *final_complete_procedure* function.

In [107]:
#final_complete_procedure(shanghai_new, wrong_names_shanghai)
#final_complete_procedure(cwur_new, wrong_names_cwur)
#final_complete_procedure(times_new, [])

In [108]:
list_shanghai=final_complete_procedure(shanghai, wrong_names_shanghai)

In [109]:
list_shanghai.head()

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year,university_name_normalized
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005,Harvard University
1,2,University of Cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005,University of Cambridge
2,3,Stanford University,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005,Stanford University
3,4,"University of California, Berkeley",3,72.8,71.8,76.0,69.4,73.9,72.2,52.7,2005,"University of California, Berkeley"
5,6,California Institute of Technology,5,67.1,59.2,68.6,59.8,65.8,52.5,100.0,2005,California Institute of Technology


In [110]:
copy_cwur=cwur.rename(columns={'institution': 'university_name'})

In [111]:
copy_cwur.head()

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


In [112]:
list_cwur=final_complete_procedure(copy_cwur, wrong_names_cwur)

In [113]:
list_cwur.head()

,world_rank,university_name,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year,university_name_normalized
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012,Harvard University
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012,Massachusetts Institute of Technology
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012,Stanford University
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012,University of Cambridge
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012,California Institute of Technology


In [114]:
wrong_names_times=[]

In [115]:
list_times=final_complete_procedure(times, wrong_names_times)

In [116]:
list_times.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year,university_name_normalized
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011,Harvard University
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011,California Institute of Technology
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33%,37 : 63,2011,Massachusetts Institute of Technology
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,"15,596",7.8,22%,42 : 58,2011,Stanford University
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,-,94.2,"7,929",8.4,27%,45 : 55,2011,Princeton University


We create a list for each of the three DataFrames that we have just normalized with the names of the universities without repetition and a total list, sum of the three lists.

In [117]:
unique_cwur=list(list_cwur['university_name_normalized'].unique())
unique_times=list(list_times['university_name_normalized'].unique())
unique_shanghai=list(list_shanghai['university_name_normalized'].unique())
total_list=unique_cwur+unique_times+unique_shanghai
total_list

['Harvard University',
 'Massachusetts Institute of Technology',
 'Stanford University',
 'University of Cambridge',
 'California Institute of Technology',
 'Princeton University',
 'University of Oxford',
 'Yale University',
 'Columbia University',
 'University of California, Berkeley',
 'University of Chicago',
 'Cornell University',
 'University of Pennsylvania',
 'University of Tokyo',
 'Johns Hopkins University',
 'Kyoto University',
 'University of California, Los Angeles',
 'University of California, San Diego',
 'Hebrew University of Jerusalem',
 'New York University',
 'Duke University',
 'Imperial College London',
 'University of Texas at Austin',
 'University College London',
 'Osaka University',
 'Northwestern University',
 'University of Toronto',
 'University of North Carolina at Chapel Hill',
 'University of Utah',
 'University of California, Santa Barbara',
 'McGill University',
 'Carnegie Mellon University',
 'University of Southern California',
 'University of Califor

To associate to each university the number of DataFrames in which it appears, we define the dictionary *diz_university* and update it as follows: for each university name in the three DataFrames, if that name has not yet been entered in the dictionary, we enter it as a key and we assign 1 to the corresponding value. If the name of the university is already present in the dictionary, we increase the associated value by 1.

In [118]:
diz_university={}

for uni in total_list:
    if uni not in diz_university:
        diz_university[uni]=1
    else:
        diz_university[uni]+=1
        
diz_university

{'Harvard University': 3,
 'Massachusetts Institute of Technology': 3,
 'Stanford University': 3,
 'University of Cambridge': 3,
 'California Institute of Technology': 3,
 'Princeton University': 3,
 'University of Oxford': 3,
 'Yale University': 3,
 'Columbia University': 3,
 'University of California, Berkeley': 3,
 'University of Chicago': 3,
 'Cornell University': 3,
 'University of Pennsylvania': 3,
 'University of Tokyo': 3,
 'Johns Hopkins University': 3,
 'Kyoto University': 3,
 'University of California, Los Angeles': 3,
 'University of California, San Diego': 3,
 'Hebrew University of Jerusalem': 3,
 'New York University': 3,
 'Duke University': 3,
 'Imperial College London': 2,
 'University of Texas at Austin': 3,
 'University College London': 3,
 'Osaka University': 3,
 'Northwestern University': 3,
 'University of Toronto': 3,
 'University of North Carolina at Chapel Hill': 3,
 'University of Utah': 3,
 'University of California, Santa Barbara': 3,
 'McGill University': 3,

### 11. In the times ranking, compute the number of times each university appears

To calculate the number of times each university appears in the DataFrame *times* we use the `size()` function from `numpy` which in this case gives us the number of elements grouped by `groupby`, therefore the number of times each given name appears.

In [119]:
times_appearances=pd.DataFrame(list_times.groupby('university_name').size().reset_index())
times_appearances.rename(columns={0:'Appearances'}, inplace=True)
times_appearances

,university_name,Appearances
0,AGH University of Science and Technology,1
1,Aalborg University,5
2,Aalto University,5
3,Aarhus University,6
4,Aberystwyth University,5
...,...,...
813,École Normale Supérieure,6
814,École Normale Supérieure de Lyon,6
815,École Polytechnique,6
816,École Polytechnique Fédérale de Lausanne,6


### 12. Find the universities that appear at most twice in the times ranking.

In [120]:
times_appearances[times_appearances['Appearances']<=2]

,university_name,Appearances
0,AGH University of Science and Technology,1
5,Adam Mickiewicz University,1
6,Aix-Marseille University,1
7,Ajou University,1
9,Alexandru Ioan Cuza University,1
...,...,...
806,Yokohama City University,1
807,Yokohama National University,1
810,Yuan Ze University,2
811,Yıldız Technical University,1


# 13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

In [121]:
times_rank=list_times[['university_name_normalized', 'world_rank', 'year']].copy()
shanghai_rank=list_shanghai[['university_name_normalized', 'world_rank', 'year']].copy()
cwur_rank=list_cwur[['university_name_normalized', 'world_rank', 'year']].copy()

We check if the years of the three rankings are expressed with values of the same type:

In [122]:
times_rank['year'].dtype==shanghai_rank['year'].dtype==cwur_rank['year'].dtype

True

Some data of the positions in the rankings have a symbol in addition to the number. We define the *values_adjustment* function to remove these symbols so that we can compare the numeric values with each other.


*values_adjustment* takes a string as input (the vale to be fixed); in the case that the string provided is an interval (it is the position expressed as interval), the two extremes of the interval are returned, separated by the symbol "-". If the string gave as input is not a range, only the numeric value is returned, without symbols.

In [123]:
def values_adjustment(string):
    expression=interval.search(string)
    num1=expression.group('num1')
    num2=expression.group('num2')
    if (num1!='') and (num2!=''):
        return (str(num1)+'-'+str(num2))
    else:
        if num1!='':
            return str(num1)
        if num2!='':
            return str(num2)
    return np.NaN
# If for example there is '=106' we make it '106' so that '106'=='106' and not '=106'!='106'

We apply the *values_adjustment* function to the *world_rank* fields of the DataFrame that have some symbols in addition to the number value of the position in the ranking.

In [124]:
times_rank['world_rank']=times_rank['world_rank'].apply(values_adjustment)
shanghai_rank['world_rank']=shanghai_rank['world_rank'].apply(values_adjustment)

In [125]:
times_rank.tail(5)

,university_name_normalized,world_rank,year
2598,Yeungnam University,601-800,2016
2599,Yıldız Technical University,601-800,2016
2600,Yokohama City University,601-800,2016
2601,Yokohama National University,601-800,2016
2602,Yuan Ze University,601-800,2016


In [126]:
shanghai_rank.tail(5)

,university_name_normalized,world_rank,year
4868,University of Jyväskylä,401-500,2015
4873,University of Milan-Bicocca,401-500,2015
4875,University of Oklahoma,401-500,2015
4884,University of Rome II – Tor Vergata,401-500,2015
4886,University of Santiago de Compostela,401-500,2015


We begin to create a DataFrame containing the names of the universities, the year of evaluation and the position in the three different rankings using the `merge` function on the *times_rank* and *shanghai_rank* DataFrames.

In [127]:
full_rank=pd.merge(times_rank[['university_name_normalized', 'year', 'world_rank']], shanghai_rank[['university_name_normalized', 'year', 'world_rank']], on=['university_name_normalized', 'year'], suffixes=['_times', '_shanghai'])
full_rank.head()

,university_name_normalized,year,world_rank_times,world_rank_shanghai
0,Harvard University,2011,1,1
1,California Institute of Technology,2011,2,6
2,Massachusetts Institute of Technology,2011,3,3
3,Stanford University,2011,4,2
4,Princeton University,2011,5,7


In [128]:
cwur_rank['world_rank'].dtype

dtype('int64')

We convert the values of the *world_rank* field of the *cwur_rank* table from 'int64' to 'string' and rename the field so as not to confuse it with the corresponding fields of the other DataFrames used.

In [129]:
cwur_rank['world_rank']=cwur_rank['world_rank'].astype(str)
cwur_rank.rename(columns={'world_rank':'world_rank_cwur'}, inplace=True)


We use the `merge` function again to finish the construction of the *full_rank* DataFrame by also inserting the data relating to the positions of the universities in the *cwur_rank* ranking.
Finally we only display the rows of the *full_rank* table that have the same value in the *world_rank_times*, *world_rank_shanghai* and *world_rank_cwur* fields.

In [130]:
full_rank=pd.merge(full_rank[['university_name_normalized', 'year', 'world_rank_times', 'world_rank_shanghai']], cwur_rank[['university_name_normalized', 'year', 'world_rank_cwur']], on=['university_name_normalized', 'year'])
full_rank[(full_rank['world_rank_times']==full_rank['world_rank_shanghai']) & (full_rank['world_rank_shanghai']==full_rank['world_rank_cwur'])]

,university_name_normalized,year,world_rank_times,world_rank_shanghai,world_rank_cwur
80,Stanford University,2013,2,2,2
